In [1]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import requests as r
import calendar
import MySQLdb

### Functions

In [2]:
##regex to find news on HTML
pattern_news = re.compile('N[1-9]')
##then crawler have to use more than one strategy to get data
pattern_title = re.compile('[1-9] -')

d = {
    'janeiro': 1,
    'fevereiro': 2,
    'março': 3,
    'abril': 4,
    'maio': 5,
    'junho': 6,
    'julho': 7,
    'agosto': 8,
    'setembro': 9,
    'outubro':10,
    'novembro': 11,
    'dezembro': 12
}

def remove_html(divs):
    aux = []
    text = []
    for new in news:
        aux = ''.join(new)
        soap = BeautifulSoup(aux,"html.parser")
        text.append(soap.text)
    return text
        
        
def parse_div(divs, pattern):
    aux = []
    news = []
    for i in range(len(divs)):
        if(pattern.search(divs[i].__str__())):
            #discover postion of regex to slice string
            index = re.search(pattern, divs[i].__str__()).start() + 4
            
            aux.append(divs[i].__str__()[index:])
            for j in range(i + 1,len(divs)):
                if(pattern.search(divs[j].__str__())):
                    break
                aux.append(divs[j].__str__())

            news.append(aux)
            aux = []
            
    return news

#fill nan titles
def fill_nan_noticia_title(df):
    for i in range(len(df['noticia-titulo'])):
        if (pd.isna(df['noticia-titulo'][i]) and re.search(pattern_title, df['noticia-strong'][i].__str__())):
            df['noticia-titulo'][i] = df['noticia-strong'][i]
            


def parse_date(df):
    number_list = []
    date_list = []
    
    for k,v in enumerate(df['boletim-titulo']):
        number, date = re.split(r'[\-\|]', v)
        number = re.sub("\D", '', number)
        number_list.append(number)
        try:
            day, month, year = date.split(' de ')
            month = d[month]
        except:
            print(k)
            day, month, year = date.split('/')
        
        day = day.replace('º','')
        date_list.append('{}-{}-{}'.format(year,month,day).replace(' ',''))
        
    df['numero'] = number_list
    df['data'] = date_list


## Split title
def split_title(df):
    for k, v in enumerate(df['noticia-titulo']):
        df['noticia-titulo'][k] = re.split(r'[2-9] -|[2-9]\)', v.__str__() )
        
        
        

# Reading dataframe with all text from "boletins"

In [3]:
df = pd.read_csv('doug_crawler/fil2.csv')
df

,boletim-titulo,noticia-titulo,noticia-strong,news,news_p,news_br
0,Boletim nº 15 | 18 de outubro de 2018,1 - Comissão de Acessibilidade da UFSJ convida...,NaN,"<div>\r\n\t<p id=""N1""><strong>1 - Comissão de ...","<p class=""titulo-int2"">Boletim nº 15 | 18 de o...","<div id=""texto-int"">\r\n ..."
1,Boletim nº 12 - 15 de outubro de 2018,1) Pedidos de Acesso e Renovação ao Protocolo/...,NaN,"<div>\r\n\t<p id=""N1""><strong>1) Pedidos de Ac...","<p class=""titulo-int2"">Boletim nº 12 - 15 de o...","<div id=""texto-int"">\r\n ..."
2,Boletim nº 11 - 11 de outubro de 2018,1 - Programação de férias no SIGRH encerra na ...,NaN,NaN,"<p class=""titulo-int2"">Boletim nº 11 - 11 de o...","<div id=""texto-int"">\r\n ..."
3,Boletim nº 13 - 16 de outubro de 2018,"1 - Proex lança portal do Comunica Extensão,2 ...",A extensão na UFSJ,"<div>\r\n\t<div>\r\n\t\t<p id=""N1""><strong>1 -...","<p class=""titulo-int2"">Boletim nº 13 - 16 de o...","<div id=""texto-int"">\r\n ..."
4,Boletim nº 10 - 10 de outubro de 2018,1 - Programa de Pós-Graduação em Ciências Farm...,NaN,"<div class=""gmail_default"" style=""font-family:...","<p class=""titulo-int2"">Boletim nº 10 - 10 de o...","<div id=""texto-int"">\r\n ..."
5,Boletim nº 16 - 19 de outubro de 2018,1 - Fauf esclarece sobre pagamento das bolsas ...,NaN,"<div>\r\n\t<p id=""N1""><strong>1 - Fauf esclare...","<p class=""titulo-int2"">Boletim nº 16 - 19 de o...","<div id=""texto-int"">\r\n ..."
6,Boletim nº 9 - 9 de outubro de 2018,1 - Conep aprova Calendário Universitário de 2...,NaN,NaN,"<p class=""titulo-int2"">Boletim nº 9 - 9 de out...","<div id=""texto-int"">\r\n ..."
7,Boletim nº 14 - 17 de outubro de 2018,1- FAUF é credenciada pelo CNPq para receber e...,NaN,NaN,"<p class=""titulo-int2"">Boletim nº 14 - 17 de o...","<div id=""texto-int"">\r\n ..."
8,Boletim nº 22 - 31 de outubro de 2018,"1 - UFSJ na luta contra a violência,2 - Interd...",7 -,"<div>\r\n\t<p id=""N1""><strong>1 - UFSJ na luta...","<p class=""titulo-int2"">Boletim nº 22 - 31 de o...","<div id=""texto-int"">\r\n ..."
9,Boletim nº 19 - 24 de outubro de 2018,1 - Informativo Dipes/Progp nº 13/2018 - Cadas...,NaN,NaN,"<p class=""titulo-int2"">Boletim nº 19 - 24 de o...","<div id=""texto-int"">\r\n ..."


## Fill NaN values in news
Boletim's files dont have same structure, that's why the crawler has to use differents strategy to get news,
next code mix the values for different strategy, in that way we'll be able to represent the news in one column dataframe

In [4]:
for index in range(len(df.news)):
    if(pd.isna(df.news[index]) or len(BeautifulSoup(df.news[index]).find_all(pattern_news))>0):
        soup = BeautifulSoup(df.news_p[index],'html.parser')
    else:
        soup = BeautifulSoup(df.news[index],'html.parser')
    divs = list(soup.children)
    news = parse_div(divs, pattern_news)
    text = remove_html(news)
    if(len(text) == 0):
        soup = BeautifulSoup(df.news_br[index], 'html.parser')
        divs = list(soup.children)
        news = parse_div(divs, pattern_news)
        text = remove_html(news)
        
    df.news[index]= text


## Fill NaN values in "noticias-titulo"

Titles also can be in tag p/strong or only strong tags, in that way we needed  2 crawl strategies  to get the data, values that are NaN on "noticias-titulo" perform a search in "noticias-strong" to fill NaN

In [5]:
fill_nan_noticia_title(df)

In [6]:
df

,boletim-titulo,noticia-titulo,noticia-strong,news,news_p,news_br
0,Boletim nº 15 | 18 de outubro de 2018,1 - Comissão de Acessibilidade da UFSJ convida...,NaN,[1 - Comissão de Acessibilidade da UFSJ convid...,"<p class=""titulo-int2"">Boletim nº 15 | 18 de o...","<div id=""texto-int"">\r\n ..."
1,Boletim nº 12 - 15 de outubro de 2018,1) Pedidos de Acesso e Renovação ao Protocolo/...,NaN,[1) Pedidos de Acesso e Renovação ao Protocolo...,"<p class=""titulo-int2"">Boletim nº 12 - 15 de o...","<div id=""texto-int"">\r\n ..."
2,Boletim nº 11 - 11 de outubro de 2018,1 - Programação de férias no SIGRH encerra na ...,NaN,[1 - Programação de férias no SIGRH encerra na...,"<p class=""titulo-int2"">Boletim nº 11 - 11 de o...","<div id=""texto-int"">\r\n ..."
3,Boletim nº 13 - 16 de outubro de 2018,"1 - Proex lança portal do Comunica Extensão,2 ...",A extensão na UFSJ,[1 - Proex lança portal do Comunica Extensão\n...,"<p class=""titulo-int2"">Boletim nº 13 - 16 de o...","<div id=""texto-int"">\r\n ..."
4,Boletim nº 10 - 10 de outubro de 2018,1 - Programa de Pós-Graduação em Ciências Farm...,NaN,[1 - Programa de Pós-Graduação em Ciências Far...,"<p class=""titulo-int2"">Boletim nº 10 - 10 de o...","<div id=""texto-int"">\r\n ..."
5,Boletim nº 16 - 19 de outubro de 2018,1 - Fauf esclarece sobre pagamento das bolsas ...,NaN,[1 - Fauf esclarece sobre pagamento das bolsas...,"<p class=""titulo-int2"">Boletim nº 16 - 19 de o...","<div id=""texto-int"">\r\n ..."
6,Boletim nº 9 - 9 de outubro de 2018,1 - Conep aprova Calendário Universitário de 2...,NaN,[1 - Conep aprova Calendário Universitário de ...,"<p class=""titulo-int2"">Boletim nº 9 - 9 de out...","<div id=""texto-int"">\r\n ..."
7,Boletim nº 14 - 17 de outubro de 2018,1- FAUF é credenciada pelo CNPq para receber e...,NaN,[1- FAUF é credenciada pelo CNPq para receber ...,"<p class=""titulo-int2"">Boletim nº 14 - 17 de o...","<div id=""texto-int"">\r\n ..."
8,Boletim nº 22 - 31 de outubro de 2018,"1 - UFSJ na luta contra a violência,2 - Interd...",7 -,[1 - UFSJ na luta contra a violência\n\r\n\t\t...,"<p class=""titulo-int2"">Boletim nº 22 - 31 de o...","<div id=""texto-int"">\r\n ..."
9,Boletim nº 19 - 24 de outubro de 2018,1 - Informativo Dipes/Progp nº 13/2018 - Cadas...,NaN,[1 - Informativo Dipes/Progp nº 13/2018 - Cada...,"<p class=""titulo-int2"">Boletim nº 19 - 24 de o...","<div id=""texto-int"">\r\n ..."


## Droping Colums

Drop support columns

In [7]:
df.drop(['noticia-strong', 'news_br', 'news_p'], axis=1, inplace=True)

In [8]:
df

,boletim-titulo,noticia-titulo,news
0,Boletim nº 15 | 18 de outubro de 2018,1 - Comissão de Acessibilidade da UFSJ convida...,[1 - Comissão de Acessibilidade da UFSJ convid...
1,Boletim nº 12 - 15 de outubro de 2018,1) Pedidos de Acesso e Renovação ao Protocolo/...,[1) Pedidos de Acesso e Renovação ao Protocolo...
2,Boletim nº 11 - 11 de outubro de 2018,1 - Programação de férias no SIGRH encerra na ...,[1 - Programação de férias no SIGRH encerra na...
3,Boletim nº 13 - 16 de outubro de 2018,"1 - Proex lança portal do Comunica Extensão,2 ...",[1 - Proex lança portal do Comunica Extensão\n...
4,Boletim nº 10 - 10 de outubro de 2018,1 - Programa de Pós-Graduação em Ciências Farm...,[1 - Programa de Pós-Graduação em Ciências Far...
5,Boletim nº 16 - 19 de outubro de 2018,1 - Fauf esclarece sobre pagamento das bolsas ...,[1 - Fauf esclarece sobre pagamento das bolsas...
6,Boletim nº 9 - 9 de outubro de 2018,1 - Conep aprova Calendário Universitário de 2...,[1 - Conep aprova Calendário Universitário de ...
7,Boletim nº 14 - 17 de outubro de 2018,1- FAUF é credenciada pelo CNPq para receber e...,[1- FAUF é credenciada pelo CNPq para receber ...
8,Boletim nº 22 - 31 de outubro de 2018,"1 - UFSJ na luta contra a violência,2 - Interd...",[1 - UFSJ na luta contra a violência\n\r\n\t\t...
9,Boletim nº 19 - 24 de outubro de 2018,1 - Informativo Dipes/Progp nº 13/2018 - Cadas...,[1 - Informativo Dipes/Progp nº 13/2018 - Cada...


Number of NaN values from "Boletim-Titulo"

In [9]:
pd.isna(df['boletim-titulo']).sum()

0

## Extracting meaning values from text
Etracting boletim number and Date

In [10]:
parse_date(df)

15


In [11]:
df

,boletim-titulo,noticia-titulo,news,numero,data
0,Boletim nº 15 | 18 de outubro de 2018,1 - Comissão de Acessibilidade da UFSJ convida...,[1 - Comissão de Acessibilidade da UFSJ convid...,15,2018-10-18
1,Boletim nº 12 - 15 de outubro de 2018,1) Pedidos de Acesso e Renovação ao Protocolo/...,[1) Pedidos de Acesso e Renovação ao Protocolo...,12,2018-10-15
2,Boletim nº 11 - 11 de outubro de 2018,1 - Programação de férias no SIGRH encerra na ...,[1 - Programação de férias no SIGRH encerra na...,11,2018-10-11
3,Boletim nº 13 - 16 de outubro de 2018,"1 - Proex lança portal do Comunica Extensão,2 ...",[1 - Proex lança portal do Comunica Extensão\n...,13,2018-10-16
4,Boletim nº 10 - 10 de outubro de 2018,1 - Programa de Pós-Graduação em Ciências Farm...,[1 - Programa de Pós-Graduação em Ciências Far...,10,2018-10-10
5,Boletim nº 16 - 19 de outubro de 2018,1 - Fauf esclarece sobre pagamento das bolsas ...,[1 - Fauf esclarece sobre pagamento das bolsas...,16,2018-10-19
6,Boletim nº 9 - 9 de outubro de 2018,1 - Conep aprova Calendário Universitário de 2...,[1 - Conep aprova Calendário Universitário de ...,9,2018-10-9
7,Boletim nº 14 - 17 de outubro de 2018,1- FAUF é credenciada pelo CNPq para receber e...,[1- FAUF é credenciada pelo CNPq para receber ...,14,2018-10-17
8,Boletim nº 22 - 31 de outubro de 2018,"1 - UFSJ na luta contra a violência,2 - Interd...",[1 - UFSJ na luta contra a violência\n\r\n\t\t...,22,2018-10-31
9,Boletim nº 19 - 24 de outubro de 2018,1 - Informativo Dipes/Progp nº 13/2018 - Cadas...,[1 - Informativo Dipes/Progp nº 13/2018 - Cada...,19,2018-10-24


## Transform Title String in a List of Titles

In [12]:
re.split(r'[2-9] -|[2-9]\)',df['noticia-titulo'][1])

['1) Pedidos de Acesso e Renovação ao Protocolo/Sipac devem ser encaminhados para o Arquivo Permanente,',
 ' Netec lança edital para bolsistas de Administração e Comunicação Social (Jornalismo)']

In [13]:
split_title(df)

In [14]:
df

,boletim-titulo,noticia-titulo,news,numero,data
0,Boletim nº 15 | 18 de outubro de 2018,[1 - Comissão de Acessibilidade da UFSJ convid...,[1 - Comissão de Acessibilidade da UFSJ convid...,15,2018-10-18
1,Boletim nº 12 - 15 de outubro de 2018,[1) Pedidos de Acesso e Renovação ao Protocolo...,[1) Pedidos de Acesso e Renovação ao Protocolo...,12,2018-10-15
2,Boletim nº 11 - 11 de outubro de 2018,[1 - Programação de férias no SIGRH encerra na...,[1 - Programação de férias no SIGRH encerra na...,11,2018-10-11
3,Boletim nº 13 - 16 de outubro de 2018,"[1 - Proex lança portal do Comunica Extensão,,...",[1 - Proex lança portal do Comunica Extensão\n...,13,2018-10-16
4,Boletim nº 10 - 10 de outubro de 2018,[1 - Programa de Pós-Graduação em Ciências Far...,[1 - Programa de Pós-Graduação em Ciências Far...,10,2018-10-10
5,Boletim nº 16 - 19 de outubro de 2018,[1 - Fauf esclarece sobre pagamento das bolsas...,[1 - Fauf esclarece sobre pagamento das bolsas...,16,2018-10-19
6,Boletim nº 9 - 9 de outubro de 2018,[1 - Conep aprova Calendário Universitário de ...,[1 - Conep aprova Calendário Universitário de ...,9,2018-10-9
7,Boletim nº 14 - 17 de outubro de 2018,[1- FAUF é credenciada pelo CNPq para receber ...,[1- FAUF é credenciada pelo CNPq para receber ...,14,2018-10-17
8,Boletim nº 22 - 31 de outubro de 2018,"[1 - UFSJ na luta contra a violência,, Interd...",[1 - UFSJ na luta contra a violência\n\r\n\t\t...,22,2018-10-31
9,Boletim nº 19 - 24 de outubro de 2018,"[1 - Informativo Dipes/Progp nº 13/201, Cadas...",[1 - Informativo Dipes/Progp nº 13/2018 - Cada...,19,2018-10-24


In [15]:
con = MySQLdb.connect('localhost', 'juaumpc', 'root')
con.select_db('doug_db')
cursor = con.cursor()

In [16]:
for k, v in enumerate(df['news']):
    sqlcommand = 'INSERT INTO  doug_server_app_boletim (data_boletim, numero) VALUES ("{}", {});'.format(df['data'][k], int(df['numero'][k]))
    cursor.execute(sqlcommand)
    con.commit()

In [17]:
df.data[2]

'2018-10-11'

### Download specific 
Debug to find the reason that why crawler is unable to get the data

In [18]:
import requests
r = requests.get('https://ufsj.edu.br/ascom/boletim033.php#')

In [19]:
with open('../ufsjpage.html','w') as f:
    f.write(r.text)